In [11]:
# Install & Import

!pip install -q gnews transformers requests pandas tweepy google-colab

import pandas as pd
from gnews import GNews
import requests
import tweepy
from transformers import pipeline
from google.colab import sheets

In [3]:
# Sentiment Analyzer (HuggingFace)

sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def get_sentiment_score(text):
    """
    Returns tuple: (label, score, numeric_value)
    - label: POSITIVE/NEGATIVE
    - score: probability from model
    - numeric_value: [-1, +1]
    """
    if not text or text.strip() == "":
        return ("NEUTRAL", 0.0, 0.0)

    try:
        result = sentiment_analyzer(text[:512])[0]  # truncate long text
        label = result["label"]
        score = result["score"]

        if label == "POSITIVE":
            return (label, score, round(score, 3))   # e.g. +0.85
        elif label == "NEGATIVE":
            return (label, score, round(-score, 3))  # e.g. -0.72
        else:
            return ("NEUTRAL", score, 0.0)
    except:
        return ("NEUTRAL", 0.0, 0.0)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# Fetch Google News

def fetch_google_news(query, limit=20):
    google_news = GNews(language="en", max_results=limit)
    news_items = google_news.get_news(query)

    news_list = []
    for item in news_items:
        news_list.append([
            item.get("title"),
            item.get("published date"),
            item.get("description"),
            item.get("url")
        ])

    df_news = pd.DataFrame(news_list, columns=["title", "published_date", "description", "url"])
    return df_news

news_df = fetch_google_news("Artificial Intelligence", limit=20)
print("Google News data fetched")

Google News data fetched


In [5]:
# Fetch Twitter Data

bearer_token = "AAAAAAAAAAAAAAAAAAAAAPF24AEAAAAAqHOYYkJsvKVJi%2F%2FClSBA5jLuFIU%3D1S3bOH1dTv2atCCfQ80VGEjM1kb7nHKDewn0acFJFgZaJJomqV"
client = tweepy.Client(bearer_token)

def fetch_twitter_data_tweepy(query, limit=10):
    response = client.search_recent_tweets(query=query,
                                           tweet_fields=["created_at", "author_id"],
                                           expansions="author_id",
                                           user_fields="username",
                                           max_results=min(limit, 10))

    tweets_list = []
    users = {user["id"]: user["username"] for user in response.includes.get("users", [])}

    for tweet in response.data:
        tweets_list.append([
            tweet.created_at,
            tweet.id,
            tweet.text,
            users.get(tweet.author_id, "unknown"),
            f"https://twitter.com/{users.get(tweet.author_id, 'unknown')}/status/{tweet.id}"
        ])

    df_twitter = pd.DataFrame(tweets_list, columns=["date", "id", "content", "username", "url"])
    return df_twitter

twitter_df = fetch_twitter_data_tweepy("Artificial Intelligence", limit=10)
print("Twitter data fetched")

Twitter data fetched


In [6]:
# Assign Sentiment

def assign_sentiment(df, text_column):
    labels, scores, numeric_values = [], [], []
    for text in df[text_column].fillna("").tolist():
        label, score, numeric = get_sentiment_score(text)
        labels.append(label)
        scores.append(score)
        numeric_values.append(numeric)

    df["sentiment_label"] = labels
    df["sentiment_confidence"] = scores
    df["sentiment_value"] = numeric_values
    return df

news_df = assign_sentiment(news_df, "title")
twitter_df = assign_sentiment(twitter_df, "content")
print("Sentiment assigned with values in [-1, +1]")

Sentiment assigned with values in [-1, +1]


In [8]:
# Save Results

news_df.to_csv("news_with_sentiment.csv", index=False)
twitter_df.to_csv("twitter_with_sentiment.csv", index=False)

print("Files saved: news_with_sentiment.csv, twitter_with_sentiment.csv")

Files saved: news_with_sentiment.csv, twitter_with_sentiment.csv


In [9]:
# Preview

display(news_df.head())
display(twitter_df.head())

,title,published_date,description,url,sentiment_label,sentiment_confidence,sentiment_value
0,3 Top Artificial Intelligence (AI) Stocks to B...,"Tue, 16 Sep 2025 13:53:00 GMT",3 Top Artificial Intelligence (AI) Stocks to B...,https://news.google.com/rss/articles/CBMihAFBV...,POSITIVE,0.998454,0.998
1,The STARD-AI reporting guideline for diagnosti...,"Mon, 15 Sep 2025 10:07:22 GMT",The STARD-AI reporting guideline for diagnosti...,https://news.google.com/rss/articles/CBMiX0FVX...,NEGATIVE,0.786875,-0.787
2,Parents testify before Congress about the dang...,"Wed, 17 Sep 2025 14:32:22 GMT",Parents testify before Congress about the dang...,https://news.google.com/rss/articles/CBMioAFBV...,NEGATIVE,0.907685,-0.908
3,AI-designed viruses are here and already killi...,"Wed, 17 Sep 2025 15:14:29 GMT",AI-designed viruses are here and already killi...,https://news.google.com/rss/articles/CBMihwFBV...,NEGATIVE,0.998940,-0.999
4,Saudi Arabia Eyes Multibillion-Dollar AI Partn...,"Wed, 17 Sep 2025 06:55:00 GMT",Saudi Arabia Eyes Multibillion-Dollar AI Partn...,https://news.google.com/rss/articles/CBMixgFBV...,POSITIVE,0.978708,0.979


,date,id,content,username,url,sentiment_label,sentiment_confidence,sentiment_value
0,2025-09-17 15:37:16+00:00,1968338474265903114,"RT @SmartAIHQ: The next generation of NFTs, po...",lni1310302,https://twitter.com/lni1310302/status/19683384...,POSITIVE,0.998925,0.999
1,2025-09-17 15:37:16+00:00,1968338474064646613,"RT @mariaaureacava1: Two weeks later, the Whit...",mariaaureacava1,https://twitter.com/mariaaureacava1/status/196...,POSITIVE,0.985927,0.986
2,2025-09-17 15:37:04+00:00,1968338423967871351,RT @GeekyGadgets: Artificial Intelligence Is U...,yomuriku1987,https://twitter.com/yomuriku1987/status/196833...,POSITIVE,0.994312,0.994
3,2025-09-17 15:37:01+00:00,1968338409266852247,The world's second humanoid robot to perform a...,CyberRobooo,https://twitter.com/CyberRobooo/status/1968338...,NEGATIVE,0.985130,-0.985
4,2025-09-17 15:36:59+00:00,1968338402295898490,"RT @MediaCellPPP: Under this action plan, I am...",UsmanRahimoon55,https://twitter.com/UsmanRahimoon55/status/196...,POSITIVE,0.995431,0.995


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [10]:
# Open Interactive Sheets

sheet_news = sheets.InteractiveSheet(df=news_df)
sheet_twitter = sheets.InteractiveSheet(df=twitter_df)

https://docs.google.com/spreadsheets/d/1rR0Jrac7xyzT_2RF4r4Ic086_q34sQJIwxE2UfSp8jk/edit#gid=0


https://docs.google.com/spreadsheets/d/1FqWs3OaQyyMlGpTnqJxwNBlyto0lmbfcGqpBr4u7OLI/edit#gid=0
